In [1]:
from __future__ import print_function
from simtk.openmm import app
import simtk.openmm as mm
from simtk import unit
from sys import stdout
current_dir ='/home/sanderroet/scripts/test_show/'

gro = app.GromacsGroFile(current_dir+'vis-md.gro')
top = app.GromacsTopFile(current_dir+'topol.top', periodicBoxVectors=gro.getPeriodicBoxVectors(),
                        includeDir='/home/sanderroet/top')




In [2]:
system = top.createSystem(nonbondedMethod=app.PME, nonbondedCutoff=1.1*unit.nanometer,
        constraints=app.HBonds, rigidWater=False, ewaldErrorTolerance=0.0005)
integrator = mm.LangevinIntegrator(310*unit.kelvin, 1.0/unit.picosecond, 2.0*unit.femtoseconds)
system.addForce(mm.MonteCarloBarostat(1*unit.atmospheres, 310*unit.kelvin, 25))

print(integrator.getTemperature())
#platform = mm.Platform.getPlatformByName('CPU')

simulation =app.Simulation(top.topology, system, integrator)#,platform)
simulation.context.setPositions(gro.positions)

310.0 K


In [3]:
with open(current_dir+'minimized_checkpoint.chk', 'rb') as f:
     simulation.context.loadCheckpoint(f.read())

In [4]:
dof = 0
system = simulation.system
dofs_from_particles = 0
for i in range(system.getNumParticles()):
    if system.getParticleMass(i) > 0* unit.dalton:
        dofs_from_particles += 3
dofs_from_constraints = system.getNumConstraints()
dofs_from_motion_removers = 0
if any(type(system.getForce(i)) == mm.CMMotionRemover for i in range(system.getNumForces())):
    dofs_from_motion_removers += 3
dof = dofs_from_particles - dofs_from_constraints - dofs_from_motion_removers
print (dof) 
print (dofs_from_particles)
print (dofs_from_constraints)
print (dofs_from_motion_removers)
#print dof, "=", dofs_from_particles, "-", dofs_from_constraints, "-", dofs_from_motion_removers

53140
67683
14540
3


In [5]:
simulation.context.setVelocitiesToTemperature(310*unit.kelvin)

In [6]:
state = simulation.context.getState(getVelocities=True, getEnergy=True)
R = unit.BOLTZMANN_CONSTANT_kB * unit.AVOGADRO_CONSTANT_NA

ke = state.getKineticEnergy()
temp = 2 * ke / dof / R
print(ke)
print(temp)
print(310*unit.kelvin/2*dof*R)
print(ke/(310*unit.kelvin/2*dof*R))
print()

68407.3075906 kJ/mol
309.653678436 K
68483815.4037 J/mol
0.998882833664



In [7]:
simulation.reporters.append(app.StateDataReporter(stdout, 1, step=True, 
    time=False, potentialEnergy=False, kineticEnergy=False, totalEnergy=False, 
    temperature=True, volume=False, density=False, progress=False, 
    remainingTime=False, speed=False, separator='\t'))
simulation.reporters.append(app.PDBReporter(current_dir+'temp_test.pdb', 100))
simulation.reporters.append(app.DCDReporter(current_dir+'temp_test.dcd', 1))
print('Equilibrating...')
simulation.step(10)
state = simulation.context.getState(getVelocities=True, getEnergy=True)
ke = state.getKineticEnergy()
temp = 2 * ke / dof / R
ref_temp = temp/(310*unit.kelvin)
while ref_temp < 0.99 or ref_temp > 1.01:
    simulation.step(10)
    state = simulation.context.getState(getVelocities=True, getEnergy=True)
    ke = state.getKineticEnergy()
    temp = 2 * ke / dof / R
    ref_temp = temp/(310*unit.kelvin)

Equilibrating...
#"Step"	"Temperature (K)"
1	275.906519339
2	227.668942612
3	218.77793406
4	208.992070688
5	166.259770672
6	141.444479722
7	158.431907688
8	164.904489869
9	143.687598713
10	146.595785191
11	177.163233577
12	178.153668406
13	151.588386577
14	154.404687965
15	179.941900281
16	174.096689005
17	151.215142797
18	160.125314746
19	179.309408514
20	163.522882175
21	142.017965066
22	156.992529215
23	174.435624579
24	157.021223726
25	141.459788595
26	160.472826929
27	174.589241388
28	156.899229129
29	147.587482421
30	166.377812259
31	172.208337635
32	151.771329612
33	147.477867711
34	167.450157801
35	169.4616804
36	150.978454868
37	152.0958542
38	169.333316137
39	165.093256609
40	147.395320447
41	152.041502848
42	166.738181318
43	159.844634393
44	147.277622881
45	156.381758662
46	167.990456836
47	157.883846456
48	147.666296216
49	158.698562595
50	168.444179097
51	158.087640607
52	151.163626536
53	162.125589148
54	167.771973421
55	157.113935958
56	153.006011752
57	163.925907737
58

In [8]:
with open(current_dir+'eq_checkpoint.chk', 'wb') as f:
    f.write(simulation.context.createCheckpoint())